In [14]:
import os
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras

model = tf.keras.models.load_model('/Users/Kelly/Desktop/Test/project_28/python_scripts/nine-sensor-model/lstm')
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 100)               44000     
_________________________________________________________________
dropout_4 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_9 (Dense)              (None, 4)                 404       
Total params: 54,504
Trainable params: 54,504
Non-trainable params: 0
_________________________________________________________________


In [63]:
#loading raw data
path = 'MovementAAL/new_dataset/stream_0000'
sequences = []

for i in range (1,3):
    file_path = path + str(i) + '.csv'
    df = pd.read_csv(file_path, header=0, usecols=["delta1", "delta2", "delta3", "delta4", "delta5", "delta6", "delta7", "delta8", "delta9"])
    values = df.values
    sequences[i] = values
    
   
#sequences = np.rot90(sequences[0])
print(sequences)
print(type(sequences))


IndexError: list assignment index out of range

In [56]:
test_preds = model.predict_classes(sequences)


ValueError: in user code:

    /opt/anaconda3/envs/project_28/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1539 predict_function  *
        return step_function(self, iterator)
    /opt/anaconda3/envs/project_28/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1529 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /opt/anaconda3/envs/project_28/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1285 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /opt/anaconda3/envs/project_28/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2825 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /opt/anaconda3/envs/project_28/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:3593 _call_for_each_replica
        return fn(*args, **kwargs)
    /opt/anaconda3/envs/

In [37]:
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.utils import to_categorical

# load a dataset group, such as train or test
def load_dataset_group(group, prefix=''):
    filepath = prefix + group + '/'
    # load all 9 files as a single array
    filenames = list()
    for n in range(1,10):
        filenames += ['sensor'+ str(n) + '-' + group + '.txt']

    X = load_group(filenames, filepath)
    # load class output
    y = load_file(prefix + group + '/' + group + '-features.txt')
    return X, y

# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
	# load all train
	trainX, trainy = load_dataset_group('train', prefix)
	print(trainX.shape, trainy.shape)
	# load all test
	testX, testy = load_dataset_group('test', prefix)
	print(testX.shape, testy.shape)

	# zero-offset class values
	trainy = trainy - 1
	testy = testy - 1

	# one hot encode y
	trainy = to_categorical(trainy)
	testy = to_categorical(testy)
	print(trainX.shape, trainy.shape, testX.shape, testy.shape)

	return trainX, trainy, testX, testy

# load a list of files into a 3D array of [samples, timesteps, features]
def load_group(filenames, prefix=''):
	loaded = list()
	for name in filenames:
		data = load_file(prefix + name)
		# print("Data: ", data)
		loaded.append(data)
		# print("loaded: ", loaded)
	# stack group so that features are the 3rd dimension
	loaded = dstack(loaded)
	
	return loaded

# load a single file as a numpy array
def load_file(filepath):
	dataframe = read_csv(filepath, header=None, delim_whitespace=True)
	# print("load_file: ", dataframe.values)
	return dataframe.values

Using TensorFlow backend.


In [58]:
trainX, trainy, testX, testy = load_dataset()
print(type(testX))
print(type(testX[0]))
print(testX)
print(testX[0])


(300, 55, 9) (300, 1)
(100, 55, 9) (100, 1)
(300, 55, 9) (300, 4) (100, 55, 9) (100, 4)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
[[[ 1  1 -2 ...  1  0  3]
  [ 1  1  1 ...  1  0  3]
  [ 1  1  1 ...  1  0  3]
  ...
  [ 2  0 -1 ...  1 -1  3]
  [ 1  0 -1 ...  1 -1  2]
  [ 1  0 -1 ...  1  1  2]]

 [[ 2  1 -1 ...  0  0  1]
  [ 2  1  2 ...  1  0  0]
  [ 0  1  2 ...  1  0  0]
  ...
  [ 0  0  0 ...  1  0  0]
  [-1  0  0 ...  1 -1  0]
  [-1  0  0 ...  1 -1 -1]]

 [[-1  1  2 ...  1  0  2]
  [-1  1  2 ...  1  0  1]
  [-1  1  2 ...  1  0  0]
  ...
  [ 4  1  1 ... -1  0  2]
  [ 0 -1  1 ...  2  1  2]
  [ 1  1  0 ...  1  2  2]]

 ...

 [[ 1  1 -1 ... -1  0  1]
  [ 1  1  0 ...  0  0  2]
  [ 0 -2  0 ...  1  0  2]
  ...
  [ 1 -1  0 ...  2 -1  0]
  [ 1 -1 -1 ...  1  0  1]
  [ 1  0 -1 ...  1  0  1]]

 [[ 0 -1 -1 ...  0 -1  0]
  [ 0 -1 -1 ... -1 -1  0]
  [ 0  1  0 ... -1 -1  0]
  ...
  [ 3  0  0 ... -1 -2 -1]
  [ 1 -1 -3 ... -2 -1 -1]
  [ 1  1 -3 ... -2 -1 -1]]

 [[-1 -1 -2 ... -2 -1 -2]
  [ 0 -1 -2 